In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import akshare as ak
from datetime import datetime, timedelta
from IPython.display import Markdown

pd.set_option('display.width',3000)
pd.set_option('display.max_columns',100)
pd.set_option('display.html.table.schema',True)


In [40]:
def load_data_932(symbol):
    df = ak.stock_zh_a_hist_pre_min_em(
        symbol=symbol, start_time="09:00:00", end_time="09:32:00")
#   print(df)
    df['时间'] = pd.to_datetime(df['时间'])
    df["Open"] = df.开盘
    df["High"] = df.最高
    df["Low"] = df.最低
    df["Close"] = df.收盘
    df["Volume"] = df.成交量
    df = df.set_index('时间', drop=False)
    df["Time"] = df.index.strftime('%H:%M')
    df = df[['Time', 'Open',  'High',    'Low', 'Close', 'Volume']]
    # df = df[df.Volume.astype(float) >0]
    df = df[df.Volume > 0]
    return df

# 模块2: 指标计算


def calculate_indicators(df):
    df['ROC'] = ta.roc(close=df.Close, length=2)
    df['SLOPE'] = ta.slope(close=df.Close, length=2)
    df['CMF'] = ta.cmf(open=df.Open, high=df.High, low=df.Low,
                       close=df.Close, volume=df.Volume, length=2)  # 柴金资金流量指标
    df['OBV'] = ta.obv(close=df.Close, volume=df.Volume)  # 能量潮指标
    return df
# 模块3: 顶背离识别和背离程度计算


def identify_top_divergence(df, price_col, indicator_col):
    top_divergences = df[(df[price_col] > df[price_col].shift(1)) &
                         (df[indicator_col] < df[indicator_col].shift(1))]
    # return len(top_divergences)>0,
    # top_divergences['Divergence_Degree'] = top_divergences[price_col].diff() - top_divergences[indicator_col].diff()
    return top_divergences

# 模块4: 底背离识别和背离程度计算


def identify_bottom_divergence(df, price_col, indicator_col):
    bottom_divergences = df[(df[price_col] < df[price_col].shift(1)) &
                            (df[indicator_col] > df[indicator_col].shift(1))]
    # print("bottom",bottom_divergences)
    # bottom_divergences['Divergence_Degree'] = bottom_divergences[indicator_col].diff() - bottom_divergences[price_col].diff()
    return bottom_divergences

In [54]:
import math
degrees = 30
radians = math.radians(degrees)
math.tan(radians)
# data = load_data_932('300141')
data = load_data_932('300207')
data = load_data_932('300115')
data = calculate_indicators(data)
data['degrees'] = data['SLOPE'].apply(math.degrees)
top_roc_divergences = identify_top_divergence(data, 'CMF', 'Volume')
bottom_roc_divergences = identify_bottom_divergence(data, 'CMF', 'Volume')
print(data)
print(top_roc_divergences)
print(bottom_roc_divergences)


                      Time   Open   High    Low  Close  Volume       ROC  SLOPE       CMF      OBV   degrees
时间                                                                                                          
2024-07-16 09:26:00  09:26  12.25  12.25  12.25  12.25    4932       NaN    NaN       NaN   4932.0       NaN
2024-07-16 09:31:00  09:31  12.27  12.30  12.17  12.26   17448       NaN    NaN  0.299856  22380.0       NaN
2024-07-16 09:32:00  09:32  12.24  12.32  12.23  12.31    9040  0.489796   0.03  0.518796  31420.0  1.718873
                      Time   Open   High    Low  Close  Volume       ROC  SLOPE       CMF      OBV   degrees
时间                                                                                                          
2024-07-16 09:32:00  09:32  12.24  12.32  12.23  12.31    9040  0.489796   0.03  0.518796  31420.0  1.718873
Empty DataFrame
Columns: [Time, Open, High, Low, Close, Volume, ROC, SLOPE, CMF, OBV, degrees]
Index: []


In [88]:
# data["roc_me"] = 100 * (data.Close - data.Close.shift(1)) / data.Close.shift(1)
# data["cmf_me"] = (data.Close - data.Close.shift(1))* data.Volume
# # data["cmf_me_2"] = (data.Open+data.High+data.Low+data.Close) / data.Open.shift(1)+data.High.shift(1)+data.Low.shift(1)+data.Close.shift(1)
# print(data)

                      Time   Open   High    Low  Close  Volume       ROC       CMF      OBV    roc_me    cmf_me
时间                                                                                                             
2024-07-12 09:26:00  09:26  14.82  14.82  14.82  14.82   12322       NaN  0.000000  12322.0       NaN       NaN
2024-07-12 09:31:00  09:31  14.78  15.13  14.41  14.49   42459 -2.226721 -0.777778 -30137.0 -2.226721 -14011.47
2024-07-12 09:32:00  09:32  14.49  15.46  14.49  15.46   33856  6.694272  1.000000   3719.0  6.694272  32840.32


In [17]:

# 模块5: 主程序
def main():
    # 假设的OHLCV数据
    
    stock_symbol_list =['300141','300114','300177','300713','300391','600686','300763','002534','300641','603386']
    stock_name_list =['和顺电气','中航电测','中海达','英可瑞','长药控股','金龙汽车','锦浪科技','西子节能','正丹股份','骏亚科技']
    stock_state_list=['冲高回落','冲高回落','震荡','高举高打','冲高回落','跟风涨停','偷偷大涨','偷偷大涨','冲高回落','冲高回落']
    data_932_list =[ load_data_932(symbol) for symbol in stock_symbol_list]
    
    main_df = None
    for symbol,name,state,data in zip(stock_symbol_list,stock_name_list,stock_state_list,data_932_list):
        data['代码']=symbol
        data['名称']=name
        data['走势']=state
        data = calculate_indicators(data)
        # 识别顶背离和底背离
        
        top_roc_divergences = identify_top_divergence(data, 'CMF', 'Volume')
        bottom_roc_divergences = identify_bottom_divergence(data, 'CMF', 'Volume')

        data["top_roc_divergences"]  = len(top_roc_divergences) > 0
        data["bottom_roc_divergences"]  = len(bottom_roc_divergences) > 0
        
        if main_df is None:
            main_df = data.tail(1)
        else:
            main_df = pd.concat([main_df, data.tail(1)])    

    main_df = main_df.sort_values(by='CMF', ascending=False)

    display(Markdown(main_df.to_markdown(index=False))) 
    return main_df

    
main()

# if __name__ == "__main__":
#     main()

| Time   |   Open |   High |   Low |   Close |   Volume |   代码 | 名称     | 走势     |       ROC |       CMF |    OBV | top_roc_divergences   | bottom_roc_divergences   |
|:-------|-------:|-------:|------:|--------:|---------:|-------:|:---------|:---------|----------:|----------:|-------:|:----------------------|:-------------------------|
| 09:32  |  48.49 |  49.48 | 48.49 |   49.28 |     8621 | 300114 | 中航电测 | 冲高回落 |  2.77372  |  0.811799 |  26054 | True                  | False                    |
| 09:32  |   4.79 |   5.24 |  4.79 |    5.18 |    31468 | 300391 | 长药控股 | 冲高回落 |  9.2827   |  0.525408 | 116688 | True                  | False                    |
| 09:32  |   8.74 |   8.99 |  8.74 |    8.9  |    13179 | 300141 | 和顺电气 | 冲高回落 |  3.7296   |  0.161222 |  39732 | True                  | False                    |
| 09:32  |  17.32 |  17.82 | 17.28 |   17.66 |    24485 | 300713 | 英可瑞   | 高举高打 |  3.88235  |  0.109174 |  81909 | True                  | False                    |
| 09:32  |   9.41 |   9.74 |  9.39 |    9.7  |    79577 | 600686 | 金龙汽车 | 跟风涨停 |  1.04167  | -0.127152 | -21613 | True                  | False                    |
| 09:32  |   7.25 |   7.49 |  7.25 |    7.48 |    75798 | 300177 | 中海达   | 震荡     |  3.88889  | -0.219677 | 219803 | True                  | False                    |
| 09:32  |  13.01 |  13.4  | 13.01 |   13.25 |    15461 | 603386 | 骏亚科技 | 冲高回落 |  0.302801 | -0.503135 |  -1245 | True                  | False                    |
| 09:32  |  27.39 |  27.41 | 26.68 |   26.79 |    31654 | 300641 | 正丹股份 | 冲高回落 | -3.42466  | -0.507198 | -59978 | False                 | False                    |
| 09:32  |  10.46 |  10.46 | 10.35 |   10.37 |     8488 | 002534 | 西子节能 | 偷偷大涨 | -5.89837  | -0.814695 | -23820 | False                 | False                    |
| 09:32  |  57.25 |  58.15 | 57.23 |   57.52 |     3866 | 300763 | 锦浪科技 | 偷偷大涨 | -0.173551 | -0.831334 |  -3902 | False                 | False                    |

,Time,Open,High,Low,Close,Volume,代码,名称,走势,ROC,CMF,OBV,top_roc_divergences,bottom_roc_divergences
时间,,,,,,,,,,,,,,
2024-07-15 09:32:00,09:32,48.49,49.48,48.49,49.28,8621,300114,中航电测,冲高回落,2.773723,0.811799,26054.0,True,False
2024-07-15 09:32:00,09:32,4.79,5.24,4.79,5.18,31468,300391,长药控股,冲高回落,9.282700,0.525408,116688.0,True,False
2024-07-15 09:32:00,09:32,8.74,8.99,8.74,8.90,13179,300141,和顺电气,冲高回落,3.729604,0.161222,39732.0,True,False
2024-07-15 09:32:00,09:32,17.32,17.82,17.28,17.66,24485,300713,英可瑞,高举高打,3.882353,0.109174,81909.0,True,False
2024-07-15 09:32:00,09:32,9.41,9.74,9.39,9.70,79577,600686,金龙汽车,跟风涨停,1.041667,-0.127152,-21613.0,True,False
2024-07-15 09:32:00,09:32,7.25,7.49,7.25,7.48,75798,300177,中海达,震荡,3.888889,-0.219677,219803.0,True,False
2024-07-15 09:32:00,09:32,13.01,13.40,13.01,13.25,15461,603386,骏亚科技,冲高回落,0.302801,-0.503135,-1245.0,True,False
2024-07-15 09:32:00,09:32,27.39,27.41,26.68,26.79,31654,300641,正丹股份,冲高回落,-3.424658,-0.507198,-59978.0,False,False
2024-07-15 09:32:00,09:32,10.46,10.46,10.35,10.37,8488,002534,西子节能,偷偷大涨,-5.898367,-0.814695,-23820.0,False,False


In [18]:
df2 = main()


| Time   |   Open |   High |   Low |   Close |   Volume |   代码 | 名称     | 走势     |       ROC |       CMF |    OBV | top_roc_divergences   | bottom_roc_divergences   |
|:-------|-------:|-------:|------:|--------:|---------:|-------:|:---------|:---------|----------:|----------:|-------:|:----------------------|:-------------------------|
| 09:32  |  48.49 |  49.48 | 48.49 |   49.28 |     8621 | 300114 | 中航电测 | 冲高回落 |  2.77372  |  0.811799 |  26054 | True                  | False                    |
| 09:32  |   4.79 |   5.24 |  4.79 |    5.18 |    31468 | 300391 | 长药控股 | 冲高回落 |  9.2827   |  0.525408 | 116688 | True                  | False                    |
| 09:32  |   8.74 |   8.99 |  8.74 |    8.9  |    13179 | 300141 | 和顺电气 | 冲高回落 |  3.7296   |  0.161222 |  39732 | True                  | False                    |
| 09:32  |  17.32 |  17.82 | 17.28 |   17.66 |    24485 | 300713 | 英可瑞   | 高举高打 |  3.88235  |  0.109174 |  81909 | True                  | False                    |
| 09:32  |   9.41 |   9.74 |  9.39 |    9.7  |    79577 | 600686 | 金龙汽车 | 跟风涨停 |  1.04167  | -0.127152 | -21613 | True                  | False                    |
| 09:32  |   7.25 |   7.49 |  7.25 |    7.48 |    75798 | 300177 | 中海达   | 震荡     |  3.88889  | -0.219677 | 219803 | True                  | False                    |
| 09:32  |  13.01 |  13.4  | 13.01 |   13.25 |    15461 | 603386 | 骏亚科技 | 冲高回落 |  0.302801 | -0.503135 |  -1245 | True                  | False                    |
| 09:32  |  27.39 |  27.41 | 26.68 |   26.79 |    31654 | 300641 | 正丹股份 | 冲高回落 | -3.42466  | -0.507198 | -59978 | False                 | False                    |
| 09:32  |  10.46 |  10.46 | 10.35 |   10.37 |     8488 | 002534 | 西子节能 | 偷偷大涨 | -5.89837  | -0.814695 | -23820 | False                 | False                    |
| 09:32  |  57.25 |  58.15 | 57.23 |   57.52 |     3866 | 300763 | 锦浪科技 | 偷偷大涨 | -0.173551 | -0.831334 |  -3902 | False                 | False                    |

In [26]:
# print(df2.columns)
print(df2[df2.top_roc_divergences.astype(bool) == True])

                      Time   Open   High    Low  Close  Volume      代码    名称    走势       ROC       CMF       OBV  top_roc_divergences  bottom_roc_divergences
时间                                                                                                                                                           
2024-07-15 09:32:00  09:32  48.49  49.48  48.49  49.28    8621  300114  中航电测  冲高回落  2.773723  0.811799   26054.0                 True                   False
2024-07-15 09:32:00  09:32   4.79   5.24   4.79   5.18   31468  300391  长药控股  冲高回落  9.282700  0.525408  116688.0                 True                   False
2024-07-15 09:32:00  09:32   8.74   8.99   8.74   8.90   13179  300141  和顺电气  冲高回落  3.729604  0.161222   39732.0                 True                   False
2024-07-15 09:32:00  09:32  17.32  17.82  17.28  17.66   24485  300713   英可瑞  高举高打  3.882353  0.109174   81909.0                 True                   False
2024-07-15 09:32:00  09:32   9.41   9.74   9.39   9.

In [31]:

# 模块5: 主程序
def main():
    # 假设的OHLCV数据
    
    stock_symbol_list =['300141','300114','300177','300713','300391','600686','300763','002534','300641','603386']
    stock_name_list =['和顺电气','中航电测','中海达','英可瑞','长药控股','金龙汽车','锦浪科技','西子节能','正丹股份','骏亚科技']
    stock_state_list=['冲高回落','冲高回落','震荡','高举高打','冲高回落','跟风涨停','偷偷大涨','偷偷大涨','冲高回落','冲高回落']
    data_932_list =[ load_data_932(symbol) for symbol in stock_symbol_list]
    
    main_df = None
    for symbol,name,state,data in zip(stock_symbol_list,stock_name_list,stock_state_list,data_932_list):
        data['代码']=symbol
        data['名称']=name
        data['走势']=state
        data = calculate_indicators(data)
        # 识别顶背离和底背离
        
        top_roc_divergences = identify_top_divergence(data, 'CMF', 'Volume')
        bottom_roc_divergences = identify_bottom_divergence(data, 'CMF', 'Volume')

        data["top_roc_divergences"]  = len(top_roc_divergences) > 0
        data["bottom_roc_divergences"]  = len(bottom_roc_divergences) > 0
        
        if main_df is None:
            main_df = data.tail(1)
        else:
            main_df = pd.concat([main_df, data.tail(1)])    

    main_df = main_df.sort_values(by='CMF', ascending=False)

    # display(Markdown(main_df.to_markdown(index=False))) 
    return main_df

    
# main()

# if __name__ == "__main__":
#     main()
df2 = main()
print(df2[df2.top_roc_divergences.astype(bool) == True])

                      Time   Open   High    Low  Close  Volume      代码    名称    走势       ROC       CMF       OBV  top_roc_divergences  bottom_roc_divergences
时间                                                                                                                                                           
2024-07-15 09:32:00  09:32  48.49  49.48  48.49  49.28    8621  300114  中航电测  冲高回落  2.773723  0.811799   26054.0                 True                   False
2024-07-15 09:32:00  09:32   4.79   5.24   4.79   5.18   31468  300391  长药控股  冲高回落  9.282700  0.525408  116688.0                 True                   False
2024-07-15 09:32:00  09:32   8.74   8.99   8.74   8.90   13179  300141  和顺电气  冲高回落  3.729604  0.161222   39732.0                 True                   False
2024-07-15 09:32:00  09:32  17.32  17.82  17.28  17.66   24485  300713   英可瑞  高举高打  3.882353  0.109174   81909.0                 True                   False
2024-07-15 09:32:00  09:32   9.41   9.74   9.39   9.

In [37]:
start_date = datetime.now() - timedelta(days=30)
start_date_str = start_date.strftime('%Y%m%d')
df =ak.stock_zh_a_hist(symbol='300141', start_date=start_date_str, adjust='')

print(df)

            日期    开盘    收盘    最高    最低     成交量           成交额     振幅    涨跌幅   涨跌额    换手率
0   2024-06-17  7.41  8.05  8.21  7.22  187855  1.453822e+08  12.68   3.07  0.24   7.45
1   2024-06-18  7.89  7.88  7.96  7.74  123321  9.667215e+07   2.73  -2.11 -0.17   4.89
2   2024-06-19  7.75  7.55  7.89  7.55   97610  7.483563e+07   4.31  -4.19 -0.33   3.87
3   2024-06-20  7.55  7.58  7.88  7.46   92094  7.059265e+07   5.56   0.40  0.03   3.65
4   2024-06-21  7.39  7.38  7.64  7.30   57440  4.282411e+07   4.49  -2.64 -0.20   2.28
5   2024-06-24  7.28  7.28  7.49  7.12   56127  4.122513e+07   5.01  -1.36 -0.10   2.22
6   2024-06-25  7.28  7.17  7.40  7.02   55683  3.999976e+07   5.22  -1.51 -0.11   2.21
7   2024-06-26  7.08  7.33  7.33  6.95   47904  3.435540e+07   5.30   2.23  0.16   1.90
8   2024-06-27  7.39  7.36  7.61  7.24   64443  4.756357e+07   5.05   0.41  0.03   2.55
9   2024-06-28  7.68  8.21  8.83  7.59  361984  3.127146e+08  16.85  11.55  0.85  14.35
10  2024-07-01  8.11  7.90  8.26

In [59]:
import math
degrees = 30
radians = math.radians(degrees)
math.tan(radians)


radians = math.degrees(-0.25)
radians


-14.32394487827058

In [32]:
df111 =ak.stock_zh_a_spot_em()
df.columns

Index(['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率'], dtype='object')

0.5773502691896257

In [7]:
stock_info = ak.stock_individual_info_em(symbol="300397")
print(stock_info)
print(stock_info.iloc[4])
total_mv = stock_info.iloc[4]['value'] / (10000 * 10000)  # 总市值
print(total_mv)

   item          value
0  股票代码         300397
1  股票简称           天和防务
2   总股本    517636745.0
3   流通股    405505408.0
4   总市值   4472381476.8
5  流通市值  3503566725.12
6    行业           通信设备
7  上市时间       20140910
item              总市值
value    4472381476.8
Name: 4, dtype: object
44.723814768000004


In [9]:
df = ak.stock_zh_a_daily(symbol='sz300397', start_date="20240701", end_date="20240719")
print(df)


         date  open  high   low  close      volume  outstanding_share  turnover
0  2024-07-01  9.21  9.30  9.00   9.29   9995220.0        405505408.0  0.024649
1  2024-07-02  9.34  9.57  9.22   9.38  14222992.0        405505408.0  0.035075
2  2024-07-03  9.27  9.32  9.10   9.13   8676677.0        405505408.0  0.021397
3  2024-07-04  9.10  9.21  8.89   8.90   8559593.0        405505408.0  0.021108
4  2024-07-05  8.90  9.04  8.76   9.00   8271389.0        405505408.0  0.020398
5  2024-07-08  8.99  8.99  8.69   8.71   7591432.0        405505408.0  0.018721
6  2024-07-09  8.69  8.90  8.53   8.86  10535187.0        405505408.0  0.025980
7  2024-07-10  8.92  8.99  8.77   8.88   8144458.0        405505408.0  0.020085
8  2024-07-11  9.04  9.14  8.95   9.12   9029491.0        405505408.0  0.022267
9  2024-07-12  9.13  9.15  8.88   8.91   8240426.0        405505408.0  0.020321
10 2024-07-15  9.02  9.15  8.80   8.83  10290343.0        405505408.0  0.025377
11 2024-07-16  8.72  8.84  8.65   8.83  

/root/anaconda3/envs/sequoia39/lib/python3.9/site-packages/akshare/stock/stock_zh_a_sina.py:198: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df.fillna(method="ffill", inplace=True)


In [11]:
405505408/1060898000

0.38222845928637816

In [12]:
df = ak.stock_zh_a_hist(symbol='300397', start_date="20240701", end_date="20240719", adjust='')
print(df)

            日期    开盘    收盘    最高    最低     成交量           成交额    振幅   涨跌幅   涨跌额   换手率
0   2024-07-01  9.21  9.29  9.30  9.00   99952  9.159037e+07  3.26  0.98  0.09  2.46
1   2024-07-02  9.34  9.38  9.57  9.22  142230  1.337113e+08  3.77  0.97  0.09  3.51
2   2024-07-03  9.27  9.13  9.32  9.10   86767  7.946484e+07  2.35 -2.67 -0.25  2.14
3   2024-07-04  9.10  8.90  9.21  8.89   85596  7.733892e+07  3.50 -2.52 -0.23  2.11
4   2024-07-05  8.90  9.00  9.04  8.76   82714  7.376898e+07  3.15  1.12  0.10  2.04
5   2024-07-08  8.99  8.71  8.99  8.69   75914  6.663581e+07  3.33 -3.22 -0.29  1.87
6   2024-07-09  8.69  8.86  8.90  8.53  105352  9.197133e+07  4.25  1.72  0.15  2.60
7   2024-07-10  8.92  8.88  8.99  8.77   81445  7.255074e+07  2.48  0.23  0.02  2.01
8   2024-07-11  9.04  9.12  9.14  8.95   90295  8.191076e+07  2.14  2.70  0.24  2.23
9   2024-07-12  9.13  8.91  9.15  8.88   82404  7.402795e+07  2.96 -2.30 -0.21  2.03
10  2024-07-15  9.02  8.83  9.15  8.80  102903  9.201860e+07  3.9

In [15]:
df =ak.stock_zh_a_spot_em()
print(df.columns)
# print(df)
df.drop(columns=['序号'],inplace=True)
df.drop(columns=['涨跌额'],inplace=True)
print(df.columns)
SPOT_EM_COLUMNS =[
  "code",
  "name",
  "price",
  "pct",
  "volume",
  "amount",
  "amplitude",
  "high",
  "low",
  "open",
  "close_yesterday",
  "volume_ratio",
  "turnover",
  "pe",
  "pb",
  "total_market_capitalization",
  "circulating_market_capitalization",
  "rate_of_increase",
  "5_minute_change",
  "60_day_pct",
  "ytd_pct"
]
print(SPOT_EM_COLUMNS)
df.columns = list(SPOT_EM_COLUMNS)
# df =df[SPOT_EM_COLUMNS]
print(df.columns)

Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高', '最低', '今开', '昨收', '量比', '换手率', '市盈率-动态', '市净率', '总市值', '流通市值', '涨速', '5分钟涨跌', '60日涨跌幅', '年初至今涨跌幅'], dtype='object')
Index(['代码', '名称', '最新价', '涨跌幅', '成交量', '成交额', '振幅', '最高', '最低', '今开', '昨收', '量比', '换手率', '市盈率-动态', '市净率', '总市值', '流通市值', '涨速', '5分钟涨跌', '60日涨跌幅', '年初至今涨跌幅'], dtype='object')
['code', 'name', 'price', 'pct', 'volume', 'amount', 'amplitude', 'high', 'low', 'open', 'close_yesterday', 'volume_ratio', 'turnover', 'pe', 'pb', 'total_market_capitalization', 'circulating_market_capitalization', 'rate_of_increase', '5_minute_change', '60_day_pct', 'ytd_pct']
        code  name  price     pct    volume        amount  amplitude   high    low   open  close_yesterday  volume_ratio  turnover       pe    pb  total_market_capitalization  circulating_market_capitalization  rate_of_increase  5_minute_change  60_day_pct  ytd_pct
0     301552  N科力装  69.20  130.67  106037.0  7.359846e+08      30.67  75.00  65.80  70.22  